# Industry Experience Iteration 3

##  Activity: Sustainability rank for house pricing, green space, transportation, CO2 emission data set



In [1]:
# pip install geopandas


In [2]:
from pathlib import Path
from ipykernel import get_connection_file

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
DATA_DIR = '../data'

In [4]:
# Loading the house price dataset
melbourne_data = pd.read_csv(DATA_DIR + '/' + 'MELBOURNE_HOUSE_PRICES_LESS_CLEAN.csv')

In [5]:
# Example of user inputs (these would be dynamic in a real application)
user_input = {
    'type': 'h',  # house or unit
    'rooms': 3,   # preferred number of rooms # make sure user does not input any number less than 3
    'distance': 10,  # preferred distance # make sure user does not input any number less than 5
    'affordable': True,  # if affordability should be maximized # preferred input: True/False
    'prefer_parks': True,  # if park preferences are included # preferred input: True/False
    'prefer_bus': True,
    'prefer_carpark': True,
    'prefer_good_air_quality_low_co2_emission': True,
    'prefer_less_crime': True
}

In [6]:
# Filter data based on user inputs
filtered_data = melbourne_data[
    (melbourne_data['Type'] == user_input['type']) &
    (melbourne_data['Rooms'].between(user_input['rooms'] - 2, user_input['rooms'] + 2)) &
    (melbourne_data['Distance'].between(user_input['distance'] - 5, user_input['distance'] + 5)) &
    (melbourne_data['Distance'] > 4)
]

# If affordability is preferred, filter for prices below the median
if user_input['affordable']:
    median_price = filtered_data['Price'].median()
    filtered_data = filtered_data[filtered_data['Price'] <= median_price]


### Aggregate Data by Suburb

Now that we’ve filtered the data, we can group it by suburb to compute metrics like the number of houses, average price, and average distance.

In [7]:
# Aggregate data by suburb
suburb_rank = filtered_data.groupby('Suburb').agg(
    HouseCount=('Suburb', 'size'),  # Count of houses
    AveragePrice=('Price', 'mean'),  # Average house price
    AverageDistance=('Distance', 'mean')  # Average distance
).reset_index()

### Calculate Ranks for Each Suburb
For house count, higher numbers should get a better rank (lower number), while for prices and distance, lower values should get a better rank.

In [8]:
# Rank the suburbs by house count, average price, and average distance
suburb_rank['Rank_HouseCount'] = suburb_rank['HouseCount'].rank(ascending=False, method='min')  # More houses -> higher rank
suburb_rank['Rank_AveragePrice'] = suburb_rank['AveragePrice'].rank(ascending=True, method='min')  # Lower price -> higher rank
suburb_rank['Rank_AverageDistance'] = suburb_rank['AverageDistance'].rank(ascending=True, method='min')  # Closer distance -> higher rank

# Total rank (sum of ranks across metrics)
suburb_rank['TotalRank'] = suburb_rank['Rank_HouseCount'] + suburb_rank['Rank_AveragePrice'] + suburb_rank['Rank_AverageDistance']

# Sort suburbs by their total rank
suburb_rank = suburb_rank.sort_values('TotalRank').reset_index(drop=True)

### Incorporating Park Preferences

In [9]:
# Load the parks data
parks_reserves_data = pd.read_csv(DATA_DIR + '/' + 'Green_parks_merge.csv')

# Aggregate the number of parks and total park area per suburb
parks_summary = parks_reserves_data.groupby('suburb').agg(
    NumberOfParks=('name', 'nunique'),  # Unique parks
    TotalParkArea=('area', 'sum')  # Total park area
).reset_index()

# Rank the suburbs based on parks
parks_summary['Rank_NumberOfParks'] = parks_summary['NumberOfParks'].rank(ascending=False, method='min')  # More parks -> higher rank
parks_summary['Rank_TotalParkArea'] = parks_summary['TotalParkArea'].rank(ascending=False, method='min')  # Larger park area -> higher rank

# Combine the ranks into an overall parks rank
parks_summary['OverallParksRank'] = (parks_summary['Rank_NumberOfParks'] + parks_summary['Rank_TotalParkArea']) / 2

# Merge the parks rank with the main suburb ranking data
suburb_rank = suburb_rank.merge(parks_summary, left_on='Suburb', right_on='suburb', how='left')

# If user prefers parks, adjust the total rank by combining with the parks rank
if user_input['prefer_parks']:
    suburb_rank['TotalRank'] += suburb_rank['OverallParksRank']

# Sort suburbs by the new total rank
suburb_rank = suburb_rank.sort_values('TotalRank').reset_index(drop=True)


### Iteration 2 Dataset integration : Transport Data - Bus Stop and car parking

Loading the Datasets.
Bus Stop Data: We will aggregate the data by suburb, calculating the number of bus stops and routes in each suburb.

Train Car Park Data: This dataset contains information on car park availability at train stations. We'll aggregate this data by suburb or train station, depending on the structure of the data.

In [10]:
# Load the bus stop and train car park datasets
bus_stop_data = pd.read_csv(DATA_DIR + '/' + 'ptv_metro_bus_stop_cleaned.csv')
train_carpark_data = pd.read_csv(DATA_DIR + '/' + 'ptv_train_carpark_cleaned.csv')

# Example: Displaying first few rows to understand the structure
print(bus_stop_data.head())
print(train_carpark_data.head())

   stop_id   latitude                                          stop_name  \
0    10960 -37.718319              Torbay St/Greensborough Hwy (Macleod)   
1    11016 -37.718029              Torbay St/Greensborough Hwy (Macleod)   
2    11017 -37.721624  Simpson Army Barracks/Greensborough Hwy (Macleod)   
3    11018 -37.723763         Strathallan Rd/Greensborough Hwy (Macleod)   
4    11019 -37.726603               Oban Way/Greensborough Hwy (Macleod)   

    longitude ticketzone routes_using_stop   suburb  postcode  
0  145.081460          2               513  MACLEOD      3085  
1  145.081354          2               513  MACLEOD      3085  
2  145.080739          2               513  MACLEOD      3085  
3  145.080423          2               513  MACLEOD      3085  
4  145.079896          2               513  MACLEOD      3085  
   commuter_capacity   station_name     suburb  postcode
0                182  Keilor Plains  St Albans      3021
1                186  Keilor Plains  St Alban

### Process the Bus Stop Data

We need to aggregate the bus stop data by suburb to understand how many bus stops and unique routes serve each suburb.

In [11]:
# Aggregate bus data by suburb
bus_service_summary = bus_stop_data.groupby('suburb').agg(
    NumberOfBusStops=('stop_id', 'size'),  # Number of bus stops
    UniqueBusRoutes=('routes_using_stop', lambda x: len(set(",".join(x).split(','))))  # Count of unique bus routes
).reset_index()

# Merge bus data with the existing suburb rankings
suburb_rank = suburb_rank.merge(bus_service_summary, left_on='Suburb', right_on='suburb', how='left')

# Replace missing values with 0 for suburbs with no bus service
suburb_rank['NumberOfBusStops'].fillna(0, inplace=True)
suburb_rank['UniqueBusRoutes'].fillna(0, inplace=True)


/tmp/ipykernel_54533/3988623181.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  suburb_rank['NumberOfBusStops'].fillna(0, inplace=True)
/tmp/ipykernel_54533/3988623181.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

### Process the train station car parks Data

We will process the train car park data, potentially aggregating it by suburb or train station. This will allow us to factor car park availability into the suburb ranking.

In [12]:
# Assuming train_carpark_data contains 'suburb' and 'carpark_capacity' columns

# Aggregate train car park data by suburb (if needed)
carpark_summary = train_carpark_data.groupby('suburb').agg(
    TotalCarparkCapacity=('commuter_capacity', 'sum')  # Total car park capacity in the suburb
).reset_index()

# Merge car park data with the suburb ranking
suburb_rank = suburb_rank.merge(carpark_summary, left_on='Suburb', right_on='suburb', how='left')

# Replace missing values with 0 for suburbs with no train car parks
suburb_rank['TotalCarparkCapacity'].fillna(0, inplace=True)


/tmp/ipykernel_54533/1093204775.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  suburb_rank['TotalCarparkCapacity'].fillna(0, inplace=True)


### Adjust the TotalRank:

In [13]:
# Rank suburbs based on bus services
suburb_rank['Rank_BusStops'] = suburb_rank['NumberOfBusStops'].rank(ascending=False, method='min')
suburb_rank['Rank_BusRoutes'] = suburb_rank['UniqueBusRoutes'].rank(ascending=False, method='min')

# Rank suburbs based on train car park capacity
suburb_rank['Rank_CarparkCapacity'] = suburb_rank['TotalCarparkCapacity'].rank(ascending=False, method='min')

# Adjust TotalRank if the user prefers bus travel
if user_input['prefer_bus']:
    suburb_rank['TotalRank'] += (suburb_rank['Rank_BusStops'] + suburb_rank['Rank_BusRoutes'])

# Adjust TotalRank if the user prefers train car parks
if user_input['prefer_carpark']:
    suburb_rank['TotalRank'] += suburb_rank['Rank_CarparkCapacity']
    
suburb_rank

,Suburb,HouseCount,AveragePrice,AverageDistance,Rank_HouseCount,Rank_AveragePrice,Rank_AverageDistance,TotalRank,suburb_x,NumberOfParks,...,Rank_TotalParkArea,OverallParksRank,suburb_y,NumberOfBusStops,UniqueBusRoutes,suburb,TotalCarparkCapacity,Rank_BusStops,Rank_BusRoutes,Rank_CarparkCapacity
0,Point Cook,297,6.627344e+05,14.7,8.0,20.0,132.0,270.5,Point Cook,8.0,...,99.0,75.5,NaN,0.0,0.0,Point Cook,150.0,1.0,1.0,33.0
1,Bundoora,438,7.375302e+05,12.1,2.0,33.0,94.0,319.0,Bundoora,4.0,...,159.0,128.0,NaN,0.0,0.0,NaN,0.0,1.0,1.0,60.0
2,St Albans,349,6.216363e+05,14.0,6.0,14.0,116.0,281.0,St Albans,3.0,...,150.0,142.0,NaN,0.0,0.0,St Albans,1588.0,1.0,1.0,1.0
3,Keilor East,247,8.146882e+05,11.7,12.0,56.0,91.0,360.0,Keilor East,3.0,...,144.0,139.0,NaN,0.0,0.0,NaN,0.0,1.0,1.0,60.0
4,Avondale Heights,180,8.093750e+05,9.5,20.0,53.0,62.0,375.0,Avondale Heights,2.0,...,167.0,178.0,NaN,0.0,0.0,NaN,0.0,1.0,1.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Huntingdale,22,9.405488e+05,12.3,99.0,115.0,96.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,0.0,1.0,1.0,60.0
137,Ormond,37,1.013263e+06,11.4,84.0,138.0,88.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,Ormond,117.0,1.0,1.0,40.0
138,Hughesdale,28,9.606401e+05,12.3,91.0,124.0,96.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,Hughesdale,102.0,1.0,1.0,43.0
139,Moorabbin,56,9.695321e+05,14.3,65.0,126.0,124.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,Moorabbin,30.0,1.0,1.0,55.0


### CO2 emission integration 

In [14]:
# Load the CO2 emission dataset
co2_emission_data = pd.read_csv(DATA_DIR + '/' + 'total_co2_emission_by_suburb.csv')

# # Display the first few rows of the dataset to examine the content
# print(co2_emission_data.head())

# Create a list to store the suburb names
suburb_names = co2_emission_data['suburb'].tolist()

# # Display the list of suburb names
# print(suburb_names)

# Check if the user prefers low CO2 emission suburbs
if user_input.get('prefer_good_air_quality_low_co2_emission', True):  # Ensure the key exists and is set to True
    # Reduce 20 from TotalRank for suburbs that match with the suburb_names list
    suburb_rank.loc[suburb_rank['Suburb'].isin(suburb_names), 'TotalRank'] -= 20



### Crime Dataset

In [15]:
# Load the cleaned crime score data
crime_score_data = pd.read_csv(DATA_DIR + '/' + 'cleaned_suburb_crime_score.csv')

# Ensure both datasets have standardized suburb names (uppercase and stripped of extra spaces)
suburb_rank['Suburb'] = suburb_rank['Suburb'].str.upper().str.strip()
crime_score_data['Suburb Name'] = crime_score_data['Suburb Name'].str.upper().str.strip()

# Check which suburbs are in suburb_rank but not in crime_score_data
missing_suburbs = suburb_rank[~suburb_rank['Suburb'].isin(crime_score_data['Suburb Name'])]
# print("Suburbs in suburb_rank but not in crime_score_data:")
# print(missing_suburbs[['Suburb']])

# Check which suburbs are in crime_score_data but not in suburb_rank
missing_in_rank = crime_score_data[~crime_score_data['Suburb Name'].isin(suburb_rank['Suburb'])]
# print("Suburbs in crime_score_data but not in suburb_rank:")
# print(missing_in_rank[['Suburb Name']])
# # Standardize the suburb names in the loaded crime score data (make sure they're uppercase and trimmed)
# crime_score_data['Suburb Name'] = crime_score_data['Suburb Name'].str.upper().str.strip()

# Merge the crime score data with the existing suburb_rank DataFrame
suburb_rank = suburb_rank.merge(crime_score_data, left_on='Suburb', right_on='Suburb Name', how='left')

# Check for missing crime scores and fill with a high value (for missing data assume high crime score)
suburb_rank['CrimeScore'].fillna(suburb_rank['CrimeScore'].max() + 1, inplace=True)

# Normalize the crime score to a smaller scale (e.g., 0-100 range)
crime_max = suburb_rank['CrimeScore'].max()
crime_min = suburb_rank['CrimeScore'].min()

# Normalized crime score in a range (e.g., 0 to 15)
suburb_rank['NormalizedCrimeScore'] = 20 * (suburb_rank['CrimeScore'] - crime_min) / (crime_max - crime_min)

# Track the original TotalRank for later comparison
suburb_rank['OriginalTotalRank'] = suburb_rank['TotalRank']

# Adjust the TotalRank based on the normalized crime score if the user prefers less crime
if user_input.get('prefer_less_crime', True):
    suburb_rank['TotalRank'] -= suburb_rank['NormalizedCrimeScore']  # Subtract normalized crime score to favor lower crime suburbs

# Calculate how much the rank has changed due to crime score influence
suburb_rank['RankChangeDueToCrime'] = suburb_rank['OriginalTotalRank'] - suburb_rank['TotalRank']

# Sort by the updated total rank
suburb_rank = suburb_rank.sort_values('TotalRank').reset_index(drop=True)

# Select the top 5 suburbs after adjusting for crime score
top_5_suburbs = suburb_rank.head(5)

# # Print the top 5 suburbs with the updated ranks and the influence of crime score on them
# print(top_5_suburbs[['Suburb', 'TotalRank', 'CrimeScore', 'NormalizedCrimeScore', 'RankChangeDueToCrime']])

/tmp/ipykernel_54533/276970498.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  suburb_rank['CrimeScore'].fillna(suburb_rank['CrimeScore'].max() + 1, inplace=True)


In [16]:
# Recompute the TotalRank based on user preferences
suburb_rank = suburb_rank.sort_values('TotalRank').reset_index(drop=True)
# suburb_rank.head(25)

### Extract the Top Suburbs

In [17]:
# Select the top 5 suburbs based on total rank
top_5_suburbs = suburb_rank.head(5)
# print(top_5_suburbs)

In [18]:
# Select the top 5 suburbs based on total rank and explicitly create a copy
top_5_suburbs = suburb_rank.head(5).copy()

# Add a 'SerialNumber' column with values 1 through 5
top_5_suburbs['SerialNumber'] = range(1, len(top_5_suburbs) + 1)

# Preview the top 5 suburbs with the serial number
# print(top_5_suburbs[['Suburb', 'TotalRank', 'SerialNumber']])


### Load the Shapefile 

In [19]:
# Load the shapefile into a GeoDataFrame
shapefile_path = DATA_DIR + '/' + 'GDA2020/vic_localities.shp'
vic_localities_sf = gpd.read_file(shapefile_path)

# Ensure that the shapefile has the correct CRS (Coordinate Reference System) in WGS84 (EPSG:4326)
if vic_localities_sf.crs != "EPSG:4326":
    vic_localities_sf = vic_localities_sf.to_crs("EPSG:4326")

# Preview the shapefile data
# print(vic_localities_sf.head())

### Filter the top suburbs

In [20]:
# Ensure both columns are uppercase and stripped of leading/trailing spaces
vic_localities_sf['LOC_NAME'] = vic_localities_sf['LOC_NAME'].str.upper().str.strip()

# Use .loc to avoid the SettingWithCopyWarning
top_5_suburbs.loc[:, 'Suburb'] = top_5_suburbs['Suburb'].str.upper().str.strip()

# Get the list of cleaned suburb names
top_suburb_names = top_5_suburbs['Suburb'].tolist()

# Filter the shapefile GeoDataFrame to include only the top 5 suburbs
filtered_vic_localities_sf = vic_localities_sf[vic_localities_sf['LOC_NAME'].isin(top_suburb_names)]

# Preview the filtered GeoDataFrame
# print(filtered_vic_localities_sf)


 ### Merge the Rank Information with the Shapefile Data

In [21]:
# Merge the ranking data from top_5_suburbs with the filtered shapefile
# Merge based on the common suburb name ('LOC_NAME' in shapefile and 'Suburb' in rank data)
final_geo_df = filtered_vic_localities_sf.merge(top_5_suburbs, left_on='LOC_NAME', right_on='Suburb')

# Drop the 'Suburb' column from the ranking DataFrame (since 'LOC_NAME' represents the same data)
final_geo_df = final_geo_df.drop(columns=['Suburb'])

#Preview the merged data to ensure that rank information is included
print(final_geo_df['LOC_NAME'].head())


0       BUNDOORA
1      FOOTSCRAY
2    KEILOR EAST
3     POINT COOK
4      ST ALBANS
Name: LOC_NAME, dtype: object


### Export the Merged Data to a GeoJSON File

In [22]:
## Define the path to save the GeoJSON file
#output_geojson_path = 'top_5_suburbs_with_ranks.geojson'

## Export the merged GeoDataFrame to GeoJSON
#final_geo_df.to_file(output_geojson_path, driver='GeoJSON')

## Confirm the export was successful
##print(f"GeoJSON file created at: {output_geojson_path}")
